In [1]:
!pip install "numpy<2" \
            torch==2.1.2+cu121 torchvision==0.16.2+cu121 torchaudio==2.1.2+cu121 \
            transformers==4.41.2 peft==0.10.0 datasets scanpy mygene tdigest loompy optuna tensorboard \
            -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu121/torch-2.1.2%2Bcu121-cp310-cp310-linux_x86_64.whl (2200.7 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.16.2%2Bcu121-cp310-cp310-linux_x86_64.whl (6.8 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.1.2%2Bcu121-cp310-cp310-linux_x86_64.whl (3.3 MB)
  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
  Using cached peft-0.10.0-py3-none-any.whl.metadata (13 kB)
  Using cached mygene-3.2.2-py2.py3-none-any.whl.metadata (10 kB)
  Using cached tdigest-0.5.2.2-py3-none-any.whl.metadata (4.9 kB)
  Using cached loompy-3.0.8-py3-none-any.whl
  Using cached optuna-4.5.0-py3-none-any.whl.metadata (17 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached triton-2.1.0-0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.3 kB)
  Using cached regex-2025.9.18-cp31

In [2]:
import os
import time
import scanpy as sc
import pandas as pd
from datasets import load_from_disk
import sys
sys.path.append("/workspace/Projects/FM/Codes/Geneformer-main")

from geneformer import TranscriptomeTokenizer, EmbExtractor

# === Directories ===
fixed_dir = "/workspace/Projects/FM/Final data/H5AD/Visium fixed"
tokenized_dir = "/workspace/Projects/FM/Final data/tokenized_geneFormer"
output_dir = "/workspace/Projects/FM/Final data/Visium_embeddings_geneFormer"
model_dir = "/workspace/Projects/FM/Codes/Geneformer-main/Geneformer-V2-104M"

os.makedirs(tokenized_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

runtime_records = []

/opt/venv/jupyter/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# --- Loop through fixed files ---
for fname in os.listdir(fixed_dir):
    if not fname.endswith("_fixed.h5ad"):
        continue

    fixed_path = os.path.join(fixed_dir, fname)
    print(f"\n🔹 Processing {fname}")
    adata = sc.read_h5ad(fixed_path)

    start_time = time.time()

    # --- Ensure each cell/spot has a unique ID and persist to disk ---
    adata.obs["cell_id"] = adata.obs_names
    adata.write(fixed_path)  # overwrite so tokenizer sees cell_id in obs

    # --- Tokenization ---
    tokenizer = TranscriptomeTokenizer(
        custom_attr_name_dict={"cell_id": "cell_id"},  # preserve barcodes
        nproc=4,
        model_version="V2"
    )

    tokenizer.tokenize_data(
        data_directory=fixed_dir,
        output_directory=tokenized_dir,
        output_prefix=fname.replace("_fixed.h5ad", ""),
        file_format="h5ad",
        input_identifier=fname.replace(".h5ad", "").replace("_fixed", "")
    )

    tokenized_file = os.path.join(tokenized_dir, fname.replace("_fixed.h5ad", ".dataset"))

    # --- Embedding Extraction ---
    embex = EmbExtractor(
        model_type="Pretrained",
        emb_mode="cell",
        emb_layer=-1,
        max_ncells=None,
        forward_batch_size=8,
        emb_label=None,
        labels_to_plot=None,
        model_version="V2"
    )

    embs_df, embs_tensor = embex.extract_embs(
        model_directory=model_dir,
        input_data_file=tokenized_file,
        output_directory=output_dir,
        output_prefix=fname.replace("_fixed.h5ad", ""),
        output_torch_embs=True
    )

    # --- Match embeddings back to AnnData using true barcodes ---
    ds = load_from_disk(tokenized_file)
    tokenized_ids = ds["cell_id"]  # barcodes carried over from obs_names

    # ensure order consistency
    common_ids = [cid for cid in tokenized_ids if cid in adata.obs_names]
    adata_sub = adata[common_ids, :].copy()
    adata_sub = adata_sub[common_ids, :]  # reindex to match embedding order
    adata_sub.obsm["X_geneformer"] = embs_tensor.cpu().numpy()

    # --- Save annotated h5ad with embeddings ---
    out_path = os.path.join(output_dir, fname.replace("_fixed.h5ad", "_with_emb.h5ad"))
    adata_sub.write(out_path)

    # --- Runtime logging ---
    total_time = time.time() - start_time
    runtime_records.append({
        "sample": fname,
        "total_time_min": round(total_time / 60, 2),
        "cells": adata.n_obs,
        "genes": adata.n_vars
    })
    print(f"✅ Finished {fname}: {total_time/60:.2f} min")

# --- Save runtime log ---
runtime_df = pd.DataFrame(runtime_records)
runtime_df.to_csv(os.path.join(output_dir, "embedding_runtimes.csv"), index=False)
print("⏱️ Runtime summary saved.")


🔹 Processing 53430_fixed.h5ad
Tokenizing /workspace/Projects/FM/Final data/H5AD/Visium fixed/53430_fixed.h5ad


/workspace/Projects/FM/Codes/Geneformer-main/geneformer/tokenizer.py:540: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for i in adata.var["ensembl_id_collapsed"][coding_miRNA_loc]
/workspace/Projects/FM/Codes/Geneformer-main/geneformer/tokenizer.py:543: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coding_miRNA_ids = adata.var["ensembl_id_collapsed"][coding_miRNA_loc]


/workspace/Projects/FM/Final data/H5AD/Visium fixed/53430_fixed.h5ad has no column attribute 'filter_pass'; tokenizing all cells.
Creating dataset.


CLS token present in token dictionary, excluding from average.
EOS token present in token dictionary, excluding from average.
100%|██████████| 534/534 [05:40<00:00,  1.57it/s]
/tmp/ipykernel_1507/2872153868.py:61: ImplicitModificationWarning: Setting element `.obsm['X_geneformer']` of view, initializing view as actual.
  adata_sub.obsm["X_geneformer"] = embs_tensor.cpu().numpy()


✅ Finished 53430_fixed.h5ad: 6.11 min

🔹 Processing 53431_fixed.h5ad
Tokenizing /workspace/Projects/FM/Final data/H5AD/Visium fixed/53431_fixed.h5ad


...

✅ Finished 26935_fixed.h5ad: 3.30 min

🔹 Processing 53434_fixed.h5ad
Tokenizing /workspace/Projects/FM/Final data/H5AD/Visium fixed/53434_fixed.h5ad


/workspace/Projects/FM/Codes/Geneformer-main/geneformer/tokenizer.py:540: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for i in adata.var["ensembl_id_collapsed"][coding_miRNA_loc]
/workspace/Projects/FM/Codes/Geneformer-main/geneformer/tokenizer.py:543: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coding_miRNA_ids = adata.var["ensembl_id_collapsed"][coding_miRNA_loc]


/workspace/Projects/FM/Final data/H5AD/Visium fixed/53434_fixed.h5ad has no column attribute 'filter_pass'; tokenizing all cells.
Creating dataset.


CLS token present in token dictionary, excluding from average.
EOS token present in token dictionary, excluding from average.
100%|██████████| 397/397 [04:12<00:00,  1.57it/s]
/tmp/ipykernel_1507/2872153868.py:61: ImplicitModificationWarning: Setting element `.obsm['X_geneformer']` of view, initializing view as actual.
  adata_sub.obsm["X_geneformer"] = embs_tensor.cpu().numpy()


✅ Finished 53434_fixed.h5ad: 4.57 min
⏱️ Runtime summary saved.
